In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf

In [2]:
model = CNN_v3a_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3) , input_shape = (48,48,1), activation = 'relu'),
    tf.keras.layers.Conv2D(128, (3,3) , activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(256, (3,3) , activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(512, (3,3) , activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(7, activation='softmax')
])

In [3]:
model.load_weights('CNN_v3a_200epochs.h5')

## Loading a test image and preprocessing it

In [4]:
test_image = cv2.imread('testimage_cropped.jpg', cv2.IMREAD_GRAYSCALE)

In [5]:
cv2.imshow('display', test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
test_image_resized_gray = cv2.resize(test_image,(48,48))

In [7]:
cv2.imshow('display', test_image_resized_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Prediction!

In [8]:
def predict_emotion(img):
    img = cv2.resize(img, (48,48))
    prediction = model.predict(img.reshape(1,48,48,1))
    prediction = np.argmax(prediction, axis = -1)
    emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
    
    return emotions[int(prediction)]

In [9]:
predict_emotion(test_image_resized_gray)

'angry'

In [10]:
img = cv2.imread('./test/happy/PrivateTest_13103594.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (48,48))

In [11]:
img.shape

(48, 48)

In [12]:
cv2.imshow('Test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
predict_emotion(img)

'happy'

## Using HC to create a bounding box around face found in image

In [14]:
hc_path = 'C:\\Users\\Randy\\Venv\\cv\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml'

In [15]:
hc = cv2.CascadeClassifier(hc_path)

In [16]:
faces = hc.detectMultiScale(test_image)

In [17]:
for face in faces:
    x, y, w, h = face
    cv2.rectangle(test_image,(x,y),(x+w, y+h),(0,0,255),3)

In [18]:
def draw_bounding_box(img):
    #hc_path = 'C:\\Users\\Randy\\Venv\\cv\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml'
    #hc = cv2.CascadeClassifier(hc_path)
    faces = hc.detectMultiScale(img)
    for face in faces:
        x, y, w, h = face
        cv2.rectangle(img,(x,y),(x+w, y+h),(0,0,255),3)
    return img

In [19]:
cv2.imshow("Faces Detected", test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
hc_path = 'C:\\Users\\Randy\\Venv\\cv\\Lib\\site-packages\\cv2\\data'

In [38]:
cv2.imshow("Faces Detected", draw_bounding_box(test_image))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
x, y, w, h = faces[0]
cropped_face = test_image[y:y+h, x:x+w]

In [23]:
def return_cropped_face(img):
    faces = hc.detectMultiScale(img)
    for (x,y,w,h) in faces:
        img = img[y:y+h, x:x+w]
    return img

In [39]:
cv2.imshow("Cropped Face", cropped_face)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
predict_emotion(cropped_face)

'surprise'

In [26]:
video_cap = cv2.VideoCapture(0)

In [27]:
if not video_cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = video_cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = hc.detectMultiScale(gray)
    
    for (x,y,w,h) in faces:
        face = gray[y:y+h, x:x+w]
        
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, 
            predict_emotion(face), 
            (50, 50), #top left
            font, 1, 
            (255, 0, 0), #BGR 
            2, 
            cv2.LINE_4)
    # Display the resulting frame
    cv2.imshow('frame', draw_bounding_box(frame))
    if cv2.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
video_cap.release()
cv2.destroyAllWindows()

In [28]:
file = 'C:\\Users\\Randy\\GitHub\\metamo\\uploaded_images\\20210815_140224.jpg'

img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)

In [29]:
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
def return_cropped_face(img):
    """
    Reads the img, detects a face in the image and returns a cropped image with only the face
    :param img: img with a face
    :return: the cropped version of only the face
    """
    faces = hc.detectMultiScale(img)
    for (x,y,w,h) in faces:
        img = img[y:y+h, x:x+w]
    return img

In [31]:
img = return_cropped_face(img)

In [33]:
img

array([], shape=(0, 0), dtype=uint8)

In [37]:
assert img

C:\Users\Randy\AppData\Local\Temp/ipykernel_12716/1512092736.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  assert img


AssertionError: 

In [ ]:
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()